In [ ]:
import os
import re
import json
import random
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Embedding, Dense, Dropout, Concatenate, LayerNormalization
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from matplotlib import colormaps
import matplotlib.cm as cm
import seaborn as sns

# Define the MultiHeadAttention custom layer
class MultiHeadAttention(Layer):
    def __init__(self, embed_dim, num_heads, proj_dim=None, output_dim=None, return_attention_weights=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.proj_dim = proj_dim if proj_dim is not None else embed_dim // num_heads
        self.output_dim = output_dim if output_dim is not None else embed_dim
        self.return_attention_weights = return_attention_weights

        # Separate Dense layers for each head's query, key, and value projections
        self.q_dense_layers = [Dense(self.proj_dim, use_bias=False, name=f"query_projection_{i}") for i in range(num_heads)]
        self.k_dense_layers = [Dense(self.proj_dim, use_bias=False, name=f"key_projection_{i}") for i in range(num_heads)]
        self.v_dense_layers = [Dense(self.proj_dim, use_bias=False, name=f"value_projection_{i}") for i in range(num_heads)]

        # Output dense layer to combine all heads' outputs
        self.output_dense = Dense(self.output_dim, use_bias=False, activation='relu', name="output_dense")
        self.supports_masking = True  # Supports masking

    def call(self, inputs, mask):
        head_outputs = []
        attention_weights_list = []

        for i in range(self.num_heads):
            q = self.q_dense_layers[i](inputs)
            k = self.k_dense_layers[i](inputs)
            v = self.v_dense_layers[i](inputs)

            matmul_qk = tf.matmul(q, k, transpose_b=True)
            dk = tf.cast(tf.shape(k)[-1], tf.float32)

            scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

            # Add mask to the scaled tensor.
            if mask is not None:
                mask = tf.cast(mask, tf.float32)
                mask_expanded = tf.expand_dims(mask, axis=-1)
                matmul_mask = tf.matmul(mask_expanded, mask_expanded, transpose_b=True)
                scaled_attention_logits += (1 - matmul_mask) * -1e9

            attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
            # Multiply back the mask if needed
            if mask is not None:
                attention_weights *= matmul_mask
            attention_weights_list.append(attention_weights)

            attention_output = tf.matmul(attention_weights, v)
            head_outputs.append(attention_output)

        concat_heads = Concatenate(axis=-1)(head_outputs)
        output = self.output_dense(concat_heads)

        if self.return_attention_weights:
            return output, attention_weights_list
        return output

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'proj_dim': self.proj_dim,
            'output_dim': self.output_dim,
            'return_attention_weights': self.return_attention_weights,
            'supports_masking': self.supports_masking
        })
        return config

# Create a model function that accepts multiple text and one numeric input.
def create_model(max_lengths, vocab_size, embedding_dim, num_heads, proj_dim, output_dim, include_attention_weights=False):
    inputs = []
    main_outputs = []
    attention_weights_outputs = []
    
    for i, max_len in enumerate(max_lengths):
        input_layer = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True, trainable=True,
                                    name=f"embedding_{i+1}")(input_layer)
        # Apply a single layer of MultiHeadAttention
        attention_layer = MultiHeadAttention(embedding_dim, num_heads, proj_dim=proj_dim, output_dim=output_dim,
                                             return_attention_weights=include_attention_weights)
        if include_attention_weights:
            attention_output, attention_weights = attention_layer(embedding_layer)
            attention_weights_outputs.append(attention_weights)
        else:
            attention_output = attention_layer(embedding_layer)
        
        # NLP processing: Apply max pooling over the sequence length dimension
        attention_pooled_output = tf.reduce_max(attention_output, axis=1)
        main_outputs.append(attention_pooled_output)
        inputs.append(input_layer)
        
    # Process numeric input and combine with text outputs.
    numeric_input = Input(shape=(18,), dtype='float32', name="numeric_input")
    inputs.append(numeric_input)
    combined_features = Concatenate()([*main_outputs, numeric_input])
    
    # Fully connected layers.
    x = Dense(128, activation='relu', kernel_initializer='he_normal')(combined_features)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    final_output = Dense(2)(x)
    
    # Define model outputs.
    model = Model(inputs=inputs, outputs=[final_output] + attention_weights_outputs if include_attention_weights else [final_output])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
    return model

# Define file paths
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/MSA_outputs'
vocab_file = os.path.join(output_dir, 'vocab_index.json')
max_lengths_file = os.path.join(output_dir, 'max_lengths.csv')
scaler_file = os.path.join(output_dir, 'scaler.pkl')
model_weights_file = os.path.join(output_dir, 'model_fold_4.h5')
vocab_size = 2025

# Load vocabulary index
with open(vocab_file, 'r', encoding='utf-8') as f:
    word_index = json.load(f)

# Load maximum lengths for each text input
max_lengths_df = pd.read_csv(max_lengths_file)
max_lengths = max_lengths_df['Max_Length'].tolist()

# Load the scaler for numeric features
scaler = joblib.load(scaler_file)

# Create the model (set include_attention_weights=True to obtain attention weights) and load weights.
analysis_model = create_model(max_lengths, vocab_size, embedding_dim=64, num_heads=2, proj_dim=64, output_dim=32,
                              include_attention_weights=True)
analysis_model.load_weights(model_weights_file)

# Define new sample text and numeric data
new_sample_text = [
    'water', 
    'The freeze-thaw cycle test method uses the rapid freeze-thaw method. The specimens are maintained in the standard maintenance room until 24 days and then removed and immersed in water for 4 days to saturate them with water. During the freeze-thaw test, the water surface in the specimen box is higher than the upper surface of the specimen by 5 mm. The maximum temperature in the center of the specimen is controlled at (5 ± 2) ℃, and the minimum temperature is controlled at (? 18 ± 2) ℃. At any time, the temperature in the center of the specimen should not be higher than 7 ℃ and not lower than ? 20 ℃. The maximum number of freeze-thaw cycles is designed to be 100, the specimens are weighed, and their dynamic modulus of elasticity is measured after every 25 freeze-thaw cycles. The distance from the loading point to the support point is 150 mm.',
    'The microstructure of the interface transition zone under waste brick concrete as characterized by SEM is shown in Fig. 5. Before the freeze-thaw cycle, the interface transition zone region of the waste brick concrete has many hydrates, as seen in Fig. 5(a), of which the main component is calcium silicate hydrate (C-S-H). The moderately dense connections between the hydrogels contribute to the bond strength between the coarse aggregates and the cement matrix. As a result, cracks appear in the interface transition zone of WCB, the internal C-S-H content decreases, the cohesion between the cement mortar and the coarse aggregate is reduced, and the interface transition zone is significantly deteriorated after 100 freeze-thaw cycles, as shown in Fig. 5(b). In addition, a large amount of ettringite is present in the interface transition zone of NCA after 100 freeze-thaw cycles, as shown in Fig. 5(c), which leads to a decrease in the interfacial cohesion of the cement paste.',
    'The influence of waste brick coarse aggregate on the overall concrete can be described in two aspects. On the one hand, the interior of waste brick coarse aggregate is loose and porous, and the interconnected capillary pores will allow water to enter the aggregate and cause freeze-thaw damage, which is unsuitable for its frost resistance. On the other hand, there are some closed pores in the waste brick coarse aggregate. These pores can play a role similar to an air-entraining agent and improve the frost resistance of concrete. However, the pre-wetting of the aggregate in this experiment will significantly reduce the number of closed pores, so the improvement effect on the frost resistance of concrete is limited.'
]
new_sample_numeric = [0.4, 0, 487, 0, 0, 0, 195, 613, 0, 900, 15, 16.3, 0, 0, 0, 0, 100, 1]

# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[,.:]', '', text)
    return text

# Process text: tokenization, converting to index sequence and padding.
new_sample_sequences = []
for i, text in enumerate(new_sample_text):
    processed_text = preprocess_text(text)
    tokens = word_tokenize(processed_text)
    sequence = [word_index.get(word, 0) for word in tokens]
    padded_sequence = pad_sequences([sequence], maxlen=max_lengths[i], padding='post', truncating='post')[0]
    new_sample_sequences.append(padded_sequence)

# Standardize numeric data
new_sample_numeric_scaled = scaler.transform(np.array([new_sample_numeric], dtype=np.float32))

# Combine inputs (text sequences and numeric features)
new_sample_input = [np.array([seq], dtype=np.float32) for seq in new_sample_sequences] + [new_sample_numeric_scaled]

# Make predictions with the model.
outputs = analysis_model.predict(new_sample_input)

# Extract the final prediction output and the attention weight outputs for each text input
final_output = outputs[0]
sequence_outputs = outputs[1:]  # Attention weights for each text input
print("Final output:", final_output)

# Visualization functions for attention scores
def remove_zero_rows_and_columns_3d(matrix):
    """
    Remove rows and columns that are entirely zero from a 3D matrix.
    Assumes that the batch size dimension is 1.
    """
    non_zero_rows = np.any(matrix[0] != 0, axis=1)  # Find non-zero rows
    non_zero_cols = np.any(matrix[0] != 0, axis=0)  # Find non-zero columns
    cleaned_matrix = matrix[0, non_zero_rows][:, non_zero_cols]
    return cleaned_matrix[np.newaxis, :, :]

def visualize_attention_scores(attention_scores, tokens, head_index):
    """
    Visualize the attention scores for a specific head using a color-mapped text display.
    """
    # Clean up the attention matrix by removing rows/columns that are all zeros.
    cleaned_attention_scores = remove_zero_rows_and_columns_3d(attention_scores)
    
    # Compute the average attention score per token and normalize it.
    row_mean_scores = np.mean(cleaned_attention_scores[0], axis=0)
    normalized_scores = (row_mean_scores - np.min(row_mean_scores)) / (np.max(row_mean_scores) - np.min(row_mean_scores))
    
    # Get the color map
    cmap = plt.colormaps['Blues']
    color_mapped_scores = [cmap(score) for score in normalized_scores]
    
    # Set up the plot area with a sufficiently wide figure.
    fig, ax = plt.subplots(figsize=(10, 4))
    
    # Use a text wrapper for long texts.
    import textwrap
    wrapper = textwrap.TextWrapper(width=95)
    wrapped_text = wrapper.wrap(" ".join(tokens))
    
    # Set starting coordinates and line height.
    y_pos = 0
    line_height = 0.03
    
    # Draw each word with the corresponding attention color.
    for line in wrapped_text:
        words = line.split()
        x_pos = 3  # Starting x coordinate for each line
        for word in words:
            if word in tokens:
                original_idx = tokens.index(word)  # Get index in the original token list
                score = normalized_scores[original_idx]  # Get corresponding attention score
                ax.text(x_pos, y_pos, word,
                        fontsize=12, fontweight='bold', family='Times New Roman',
                        color=cmap(score), ha='left', va='center')
            x_pos += len(word) + 1  # Update x coordinate with a space between words
        y_pos -= line_height  # Move down for the next line

    ax.axis([0, 100, y_pos, 0])
    ax.axis('off')  # Hide axes

    # Define the directory where the image will be saved.
    save_dir = os.path.join(output_dir, "visualizations")
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    save_path = os.path.join(save_dir, f"attention_head_{head_index + 1}.svg")
    
    try:
        plt.savefig(save_path, format='svg', dpi=300)
        print(f"Figure saved at: {save_path}")
    except PermissionError:
        print("Permission denied. Unable to save the figure.")
    
    plt.show()

# Choose a text column to visualize attention (e.g., using the 3rd text input).
text_column_index = 3  # For the first text column, use index 1 (here we use index 3 for demonstration)
attention_matrices = outputs[text_column_index]  # Get the attention head outputs for the selected text input

# Get the tokenized words for the selected text sample.
tokens = word_tokenize(preprocess_text(new_sample_text[text_column_index - 1]))

# Loop through each head (2 heads in this case) and create visualizations.
for head_index in range(2):
    attention_scores = attention_matrices[head_index]
    print(f"Attention scores shape for head {head_index + 1}: {attention_scores.shape}")
    visualize_attention_scores(attention_scores, tokens, head_index)
